In [84]:
import pandas as pd
import numpy as np
act_train = pd.read_csv('act_train.csv')
act_test = pd.read_csv('act_test.csv')
people = pd.read_csv('people.csv')

In [18]:
act_train['people_id'] = act_train['people_id'].apply(lambda x: x.split('_')[1])
people['people_id'] = people['people_id'].apply(lambda x: x.split('_')[1])
people['group_1'] = people['group_1'].apply(lambda x:x.split(' ')[1])
people['char_2'] = people['char_2'].apply(lambda x: x.split(' ')[1])

In [85]:
act_train_lr = act_train[['people_id','outcome']]
people_lr = people[['people_id','group_1','char_2','char_38']]
train = pd.merge(act_train_lr,people_lr,how = 'left',on = 'people_id',left_index = True)
train.fillna('-999',inplace=True)
train.head()

,people_id,outcome,group_1,char_2,char_38
0,ppl_100,0,group 17304,type 2,36
0,ppl_100,0,group 17304,type 2,36
0,ppl_100,0,group 17304,type 2,36
0,ppl_100,0,group 17304,type 2,36
0,ppl_100,0,group 17304,type 2,36


In [70]:
train.fillna('-999',inplace = True)

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
lr = LogisticRegression(C=100000.0)
lr = lr.fit(train[['group_1','char_2','char_38']],act_train[['outcome']])

//anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [22]:
lr_train = lr.predict_proba(train[['group_1','char_2','char_38']])[:,1]
roc_auc_score(train.outcome,lr_train)

0.88981033649732777

In [88]:
test_lr = act_test[['activity_id','people_id']]
#test_lr['people_id'] = test_lr['people_id'].apply(lambda x: x.split('_')[1])
test_lr = pd.merge(test_lr,people_lr,how = 'left', on ='people_id',left_index = True)
test_lr.fillna('-999',inplace = True)
activity_id = test_lr.activity_id.values
test_lr.drop('activity_id',inplace = True, axis = 1) 
test_lr['outcome'] = 0
test_lr.head()

,people_id,group_1,char_2,char_38,outcome
3,ppl_100004,group 22593,type 3,76,0
3,ppl_100004,group 22593,type 3,76,0
5,ppl_10001,group 25417,type 3,90,0
5,ppl_10001,group 25417,type 3,90,0
5,ppl_10001,group 25417,type 3,90,0


In [30]:
lr_test = lr.predict_proba(test_lr[['group_1','char_2','char_38']])[:,1]
lr_test_submission = pd.DataFrame({'activity_id': act_test['activity_id'],'outcome':lr_test})


In [31]:
lr_test_submission.to_csv('selected_lr.csv',index = False)

In [53]:
train_group = train.groupby(['group_1']).mean().reset_index()
train_group_outcomes = train['outcome'].values
x = pd.merge(train[['group_1','outcome']],train_group,suffixes = ('x_',''),how = 'left',on = ['group_1'],left_index = True)
x = x['outcome']

In [59]:
x = ((x*x.shape[0]-train_group_outcomes)/(x.shape[0]-1))
x.fillna(x.mean())

4691     0.0
4691     0.0
4691     0.0
4691     0.0
4691     0.0
4691     0.0
28979    1.0
28979    1.0
15962    1.0
15962    1.0
15962    1.0
15962    1.0
15962    1.0
15962    1.0
15962    1.0
15962    1.0
15962    1.0
15962    1.0
15962    1.0
15962    1.0
15962    1.0
15962    1.0
15962    1.0
15962    1.0
15962    1.0
15962    1.0
15962    1.0
15962    1.0
15962    1.0
15962    1.0
        ... 
5064     1.0
5064     1.0
5064     1.0
5064     1.0
5064     1.0
5064     1.0
5064     1.0
5064     1.0
5064     1.0
5064     1.0
5064     1.0
5064     1.0
5064     1.0
5064     1.0
5064     1.0
5064     1.0
5064     1.0
5064     1.0
5064     1.0
5064     1.0
5064     1.0
5064     1.0
5064     1.0
5064     1.0
5064     1.0
5064     1.0
5064     1.0
5064     1.0
5064     1.0
5064     1.0
Name: outcome, dtype: float64

In [86]:
def LeaveOneOut(data1, data2, columnName, useLOO=False):
    grpOutcomes = data1.groupby(columnName).mean().reset_index()
    outcomes = data2['outcome'].values
    x = pd.merge(data2[[columnName, 'outcome']], grpOutcomes,
                 suffixes=('x_', ''),
                 how='left',
                 on=columnName,
                 left_index=True)['outcome']
    if(useLOO):
        x = ((x*x.shape[0])-outcomes)/(x.shape[0]-1)
    return x.fillna(x.mean())

In [87]:
train_3feature = pd.DataFrame()
for col in train.columns:
    if (col !='outcome' and col !='people_id'):
        train_3feature[col] = LeaveOneOut(train,train,col,True).values
train_3feature

,group_1,char_2,char_38
0,0.0,0.000000,0.000000
1,0.0,0.000000,0.000000
2,0.0,0.000000,0.000000
3,0.0,0.000000,0.000000
4,0.0,0.000000,0.000000
5,0.0,0.000000,0.000000
6,1.0,0.698855,0.727121
7,1.0,0.698855,0.727121
8,1.0,0.698855,0.731619
9,1.0,0.698855,0.731619


In [89]:
test_3feature = pd.DataFrame()
for col in train.columns:
    if (col !='outcome' and col !='people_id'):
        test_3feature[col] = LeaveOneOut(train,test_lr,col,False).values
test_3feature.head()

,group_1,char_2,char_38
0,0.0,0.698855,0.727121
1,0.0,0.698855,0.727121
2,1.0,0.698855,0.796851
3,1.0,0.698855,0.796851
4,1.0,0.698855,0.796851


In [91]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import accuracy_score,roc_auc_score

In [95]:
from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train_3feature,train['outcome'],test_size = 0.2,random_state = 1)

In [96]:
xgb1 = XGBClassifier()
xgb1 = xgb1.fit(X_train,y_train,eval_metric = 'auc')
xgb1_proba = xgb1.predict_proba(X_test)
roc_auc_score(y_test,xgb1_proba[:,1])

1.0

In [97]:
xgb1_test = xgb1.predict_proba(test_3feature)
xgb1_sub = pd.DataFrame({'activity_id':activity_id,'outcome':xgb1_test[:,1]})
#xgb1_sub.to_csv('3feature_xgb1_sub.csv',index = False)

,activity_id,outcome
0,act1_249281,0.067697
1,act2_230855,0.067697
2,act1_240724,0.999911
3,act1_83552,0.999911
4,act2_1043301,0.999911


In [103]:
xgb1_sub.head()
xgb1_sub.to_csv('3feature_xgb1_sub.csv',index = False)

In [101]:
def score(x):
    if x >0.99:
        a = 1
    elif x <0.05:
        a = 0
    else:
        a = x
    return a
xgb1_sub['outcome'] = xgb1_sub['outcome'].apply(lambda x: score(x))

In [106]:
from sklearn.grid_search import GridSearchCV
param_test1 = {
 'max_depth':list(range(3,10,2)),
 'min_child_weight':list(range(1,6,2))
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_train,y_train)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 1.00000, std: 0.00000, params: {'min_child_weight': 1, 'max_depth': 3},
  mean: 1.00000, std: 0.00000, params: {'min_child_weight': 3, 'max_depth': 3},
  mean: 1.00000, std: 0.00000, params: {'min_child_weight': 5, 'max_depth': 3},
  mean: 1.00000, std: 0.00000, params: {'min_child_weight': 1, 'max_depth': 5},
  mean: 1.00000, std: 0.00000, params: {'min_child_weight': 3, 'max_depth': 5},
  mean: 1.00000, std: 0.00000, params: {'min_child_weight': 5, 'max_depth': 5},
  mean: 1.00000, std: 0.00000, params: {'min_child_weight': 1, 'max_depth': 7},
  mean: 1.00000, std: 0.00000, params: {'min_child_weight': 3, 'max_depth': 7},
  mean: 1.00000, std: 0.00000, params: {'min_child_weight': 5, 'max_depth': 7},
  mean: 1.00000, std: 0.00000, params: {'min_child_weight': 1, 'max_depth': 9},
  mean: 1.00000, std: 0.00000, params: {'min_child_weight': 3, 'max_depth': 9},
  mean: 1.00000, std: 0.00000, params: {'min_child_weight': 5, 'max_depth': 9}],
 {'max_depth': 3, 'min_child_weight': 1

In [107]:
xgb2 = XGBClassifier( learning_rate =0.01, n_estimators=5000, max_depth=3,
 min_child_weight=10, gamma=0, subsample=0.7, colsample_bytree=0.7,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27)
xgb2 = xgb2.fit(train_3feature,train['outcome'],eval_metric = 'auc')

In [108]:
xgb2_test = xgb2.predict_proba(test_3feature)
xgb2_sub = pd.DataFrame({'activity_id':activity_id,'outcome':xgb2_test[:,1]})
xgb2_sub.to_csv('3feature_xgb2_sub.csv',index = False)